In [1]:
include("../scripts/robot.jl")

In [7]:
using Distributions
using LinearAlgebra

In [8]:
mutable struct Particle
    pose
end

In [3]:
mutable struct Mcl
    particles
    motion_noise_rate_pdf
end

In [28]:
function Mcl_(init_pose; num, motion_noise_stds)
    self = Mcl(ntuple(x->nothing, fieldcount(Mcl))...)
    self.particles =[Particle(init_pose) for i in 1:num]
    v = motion_noise_stds
    c = Diagonal([v["nn"]^2, v["no"]^2, v["on"]^2, v["oo"]^2])
    self.motion_noise_rate_pdf = MvNormal(zeros(4), c)
    return self
end

Mcl_ (generic function with 2 methods)

In [36]:
function motion_update(self::Mcl, nu, omega, time)
    print(self.motion_noise_rate_pdf.Σ)
end

motion_update (generic function with 1 method)

In [37]:
function draw(self::Mcl, ax, elems)
    xs = [p.pose[1] for p in self.particles]
    ys = [p.pose[2] for p in self.particles]
    vxs = [cos(p.pose[3]) for p in self.particles]
    vys = [sin(p.pose[3]) for p in self.particles]
    elems = vcat(elems, ax.quiver(xs, ys, vxs, vys, color="blue", alpha=0.5))
end

draw (generic function with 7 methods)

In [38]:
mutable struct EstimationAgent <: AbstractAgent
    nu
    omega
    estimator
    time_interval
end

In [39]:
function  EstimationAgent_(time_interval, nu, omega,estimator,)
    self = EstimationAgent(ntuple(x->nothing, fieldcount(EstimationAgent))...)
    self.nu = nu
    self.omega = omega
    self.estimator = estimator
    self.time_interval = time_interval
    return self
end

EstimationAgent_ (generic function with 1 method)

In [40]:
function draw(self::EstimationAgent, ax, elems)
    draw(self.estimator, ax, elems)
end

draw (generic function with 7 methods)

In [41]:
initial_pose = [0.0, 0.0, 0.0]
estimator = Mcl_(initial_pose, 100, Dict("nn"=>0.01, "no"=>0.02, "on"=>0.03, "oo"=>0.04))
a = EstimationAgent_(0.1, 0.2, 10.0/180*pi, estimator)
motion_update(estimator, 0.2, 10.0/180*pi, 0.1)

[0.0001 0.0 0.0 0.0; 0.0 0.0004 0.0 0.0; 0.0 0.0 0.0009 0.0; 0.0 0.0 0.0 0.0016]

In [43]:
rand(estimator.motion_noise_rate_pdf)

4-element Array{Float64,1}:
  0.003919016538482222
  0.006612137905653965
  0.005627358745403897
 -0.026563676826450992